In [ ]:
!pip install gurobipy
import gurobipy as gp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 25.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from gurobipy import *


In [ ]:
df = pd.read_excel('real_distances_10customers.xls', index_col = 0) # dataframe
#df = pd.read_excel('tsp_data.xls') # dataframe
df

In [ ]:
df.shape[0]

11

In [ ]:
nodes = range(df.shape[0])# List of nodes
nodes

range(0, 11)

In [ ]:
for i in nodes:
 print(i)


In [ ]:
df.iloc[0,1]

In [ ]:
d = [[df[j][i] for j in nodes] for i in nodes]# Build distance matrix
d

In [ ]:
d[10][1]


In [ ]:
tsp = Model("tsp_model")


In [ ]:
x = tsp.addVars(nodes, nodes,lb = 0,vtype = GRB.BINARY, name = 'x')
u = tsp.addVars(nodes, lb = 0,vtype = GRB.INTEGER, name = 'u')

In [ ]:
tsp.addConstrs(((quicksum(x[i,j] for j in nodes if j != i) == 1) for i in nodes), name="degree_const_1")

In [ ]:
tsp.addConstrs(((quicksum(x[i,j] for i in nodes if i != j) == 1) for j in nodes), name="degree_const_2")


In [ ]:
tsp.addConstrs(((u[i] - u[j] + len(nodes)*x[i,j] <= len(nodes) - 1) for i in nodes for j in nodes if i!=0 and j!=0),name="abc")

In [ ]:
tsp.setObjective((quicksum(d[i][j]*x[i,j] for i in nodes for j in nodes if i != j)),GRB.MINIMIZE)


In [ ]:
tsp.setParam("TimeLimit", 7200) # time limit
#tsp.setParam("OutputFlag", 0) # do not print on console


In [ ]:
tsp.update()

In [ ]:
tsp.optimize()


In [ ]:
status = tsp.status
object_Value = tsp.objVal
print("model status is: ", status)
print("Objective value is: ", object_Value)


In [ ]:
x

In [ ]:
if status !=3 and status != 4:
    for v in tsp.getVars():
        if tsp.objVal < 1e+99  and v.x!=0:
            print('%s %f'%(v.Varname,v.x))


NameError: name 'status' is not defined

In [ ]:
if status != 3 and status != 4:
 vis = []
 Sol_x = np.zeros([len(nodes), len(nodes)])
 for i in nodes:
  for j in nodes:
   if tsp.objVal < 1e+99:
    Sol_x[i,j] = x[i,j].getAttr("X")
 else:
   error_status = True
 ofvv = 1e+99
 if 1 - 0.00001 <= Sol_x[i,j] <= 1 + 0.00001:
  vis.append((i,j))


In [ ]:
df = pd.read_excel('real_distances_10customers.xls') # dataframe
#df = pd.read_excel('tsp_data.xls') # dataframe

nodes = range(df.shape[0])# List of nodes

dist = [[df[j][i] for j in nodes] for i in nodes]# Build distance matrix

#Create a new model
tsp = Model("tsp_model")

#Create variables
x = tsp.addVars(nodes, nodes, lb = 0,vtype = GRB.BINARY,name = 'x')
u = tsp.addVars(nodes, lb = 0,vtype = GRB.INTEGER,name = 'u')

#Add constraint
tsp.addConstrs(((quicksum(x[i,j] for j in nodes if j != i) == 1) for i in nodes), name = '1')
tsp.addConstrs(((quicksum(x[i,j] for i in nodes if i != j) == 1) for j in nodes), name = '2')
tsp.addConstrs(((u[i] - u[j] + len(nodes)*x[i,j] <= len(nodes) - 1) for i in nodes for j in nodes if i != j and i != 0 and j != 0), name = '3')


#set objective function
tsp.setObjective((quicksum(dist[i][j]*x[i,j] for i in nodes for j in nodes if i != j)),GRB.MINIMIZE)

tsp.setParam("TimeLimit", 120)   # time limit
#tsp.setParam("OutputFlag", 0)     # do not print on console
tsp.setParam("JSONSolDetail",1)
tsp.update()
tsp.optimize()
tsp.write('sina.json')

status = tsp.status

object_Value = tsp.objVal

print()
print("model status is: ", status)
print()
print("Objective value is: ", object_Value)


if status !=3 and status != 4:
    for v in tsp.getVars():
        if tsp.objVal < 1e+99  and v.x!=0:
            vv = v.Varname,v.x
            print('%s %f'%(v.Varname,v.x))

if status != 3 and status != 4:
    vis = []
    Sol_x = np.zeros([len(nodes), len(nodes)])
    for i in nodes:
        for j in nodes:
            if tsp.objVal < 1e+99:
                Sol_x[i,j] = x[i,j].getAttr("X")
            else:
                error_status = True
                ofvv = 1e+99
            if 1 - 0.00001 <= Sol_x[i,j] <= 1 + 0.00001:
                vis.append((i,j))


In [ ]:

visited = np.array(vis)
prt_solution = []
if visited[0][0] == 0:
    sol = [visited[0][0], visited[0][1]]
elif visited[0][0] !=0 and visited[0][1] == 0:
    sol = [visited[0][1], visited[0][0]]
else:
    print('First tuple should include depot 0')
visited = np.delete(visited,0, axis = 0)


for i in visited:
    try:
        next_ind = int(np.where(visited[:,0] == sol[-1])[0])
        sol.append(visited[next_ind][1])
        visited = np.delete(visited,next_ind, axis = 0)
    except:
        next_ind = int(np.where(visited[:,1] == sol[-1])[0])
        sol.append(visited[next_ind][0])
        visited = np.delete(visited,next_ind, axis = 0)

    if sol[0] == sol[-1]:
        sol = np.asarray(sol)
        prt_solution.append(sol)
        used = []
        for j in prt_solution:
            for k in j:
                used.append(k)
        remain = list(set(nodes) - set(used))
        if remain == []:
            break
        sol=[visited[0][0], visited[0][1]]
        visited = np.delete(visited,0, axis = 0)

print()
print('optimal tour is:', sol)

In [ ]:
print(Sol_x)
print()
print(vis)


In [ ]:

tsp.write('tsp.lp')